# Data Base

By: Javier Martínez

In [1]:
import json
import pandas as pd
from datetime import datetime
import numpy as np

### Reading Data

In [2]:
# You can safely assume that `build_dataset` is correctly implemented
def build_dataset(path):
    data = [json.loads(x) for x in open(path)]

    #================================
    def lavel_training(x,lavel=1):
        """
        Insert key in dictionary
        """
        x['training_data'] = lavel
        return x
    #================================

    target = lambda x: x.get("condition")
    N = -10000
    X_train = [lavel_training(x,lavel=1) for x in data[:N]]
    X_test = [lavel_training(x,lavel=0) for x in  data[N:]]
    y_train = [target(x) for x in X_train]
    y_test = [target(x) for x in X_test]
    # for x in X_test:
    #     del x["condition"]
    return X_train, y_train, X_test, y_test

In [3]:
# Reading Data
path = 'data/MLA_100k_checked_v3.jsonlines'
X_train, y_train, X_test, y_test = build_dataset(path)

In [4]:
# Add data
data = X_train + X_test

### Creating variables

In [5]:
# All keys
keys_list = []
for doc in data:
    for key_ in doc.keys():
        keys_list.append(key_)
    
keys_list = list(set(keys_list))

In [6]:
# Get values
data_dict = {}
for variable in keys_list :
    data_dict[variable] = list(map( lambda x: x.get(variable), data ))

all_features  = list(data_dict.keys())

Delete features:

* international_delivery_mode: All NULL.
* listing_source: All NULL ({''}).
* coverage_areas: All NULL ([])..
* differential_pricing: All NULL.
* subtitle: All NULL ({None}).
* descriptions: equal to column ID. 
* site_id : same encoding 'MLA').
* catalog_product_id: Only 11 values.
* original_price: Only 148 values.
* sub_status: Only 986 values.

In [7]:
# features dropped
features_dropped = ['international_delivery_mode',
                    'listing_source',
                    'coverage_areas',
                    'differential_pricing',
                    'subtitle',
                    'descriptions',
                    'site_id',      
                    'catalog_product_id',
                    'original_price',
                    'sub_status'   
                    ]
for feature in features_dropped:
    del data_dict[feature]

In [8]:
variables = list(data_dict.keys())

non_mercado_pago_payment_methods

In [9]:
#==========================
def non_mercado_pago_payment_methods_function(x):


    no_pago = x.get('non_mercado_pago_payment_methods')
    if (no_pago ==[]) :
        return None

    data = pd.concat( list(map( lambda x: pd.DataFrame(x,index=[0]), no_pago ) ) )
    data['id_no_pago'] = data['id'] 
    data['id'] = x.get('id')
    return data 
#==========================

pd_no_pago = list(map(non_mercado_pago_payment_methods_function,data))

In [10]:

pd_integracion = pd.concat( list(filter(lambda x: x is not None, pd_no_pago)) )

In [11]:
pd_no_pago_pivot = pd_integracion\
                        .pivot(index='id', columns='id_no_pago', values='type')\
                        .fillna(0)\
                        .replace('G',1)\
                        .replace('N',1)\
                        .replace('C',1)\
                        .replace('D',1)\
                        .reset_index(drop=False)\
                        .copy()

pd_no_pago_pivot

id_no_pago,id,MLAAM,MLABC,MLACD,MLADC,MLAMC,MLAMO,MLAMP,MLAMS,MLAOT,MLATB,MLAVE,MLAVS,MLAWC,MLAWT
0,MLA1000058677,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,MLA1000182804,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,MLA1000356806,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,MLA1000434568,0,0,0,0,0,1,0,0,1,1,0,0,1,0
4,MLA1000488061,0,0,0,0,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69436,MLA9999613215,0,0,0,0,0,1,0,0,0,1,0,0,0,0
69437,MLA9999686603,0,0,0,0,0,1,0,0,1,1,0,0,0,0
69438,MLA9999689095,0,0,0,0,0,1,0,0,0,1,0,0,1,0
69439,MLA9999718961,0,0,0,0,0,1,0,0,0,0,0,0,1,0


Creating features

In [12]:
# shipping
data_dict['local_pick_up'] = [x.get('local_pick_up') for x in data_dict['shipping']]
data_dict['free_shipping'] = [x.get('free_shipping') for x in data_dict['shipping']]
data_dict['mode'] = [x.get('mode') for x in data_dict['shipping']]

In [13]:
# seller_address
data_dict['seller_address_country'] = [x.get('country').get('id')  for x in data_dict['seller_address']]
data_dict['seller_address_state'] = [x.get('state').get('id')  for x in data_dict['seller_address']]
data_dict['seller_address_city'] = [x.get('city').get('id')  for x in data_dict['seller_address']]

In [14]:
#tags
data_dict['tags_no_list'] = [x[0] if x!=[] else 'no_tags' for x in data_dict['tags']  ]

In [15]:
# variations
data_dict['variations_new'] = [1 if x!=[] else 0 for x in data_dict['variations']  ]

In [16]:
# deal_ids
data_dict['deal_ids_new'] = [1 if x!=[] else 0 for x in data_dict['deal_ids']  ]

In [17]:
# video_id
data_dict['video_id_new'] = [1 if x!=None else 0 for x in data_dict['video_id']  ]

In [18]:
# attributes
data_dict['attributes_new'] = [1 if x!=[] else 0 for x in data_dict['attributes']  ]

### Pandas Data Frame

In [19]:
# features dropped
features_dropped = ['non_mercado_pago_payment_methods',
                    'permalink',
                    'secure_thumbnail',
                    'shipping',
                    'seller_address',
                    'pictures',
                    'thumbnail',
                    'tags',
                    'variations',
                    'deal_ids',
                    'video_id',
                    'attributes'
                    ]

for feature in features_dropped:
    del data_dict[feature]

In [20]:
#==============
def condition_format(x):
    """
    Condition boolean
    """

    if x=='new':
        return 1
    elif x=='used':
        return 0
    else:
        return x
#==============


# DataFrame
pd_data = pd.DataFrame(data_dict)
pd_data['condition_new'] = pd_data['condition'].apply(condition_format,1)

In [21]:
booleano = ['accepts_mercadopago',
            'automatic_relist',
            'local_pick_up',
            'free_shipping',
            ]

for value in booleano:
    pd_data[value] = pd_data[value].apply(lambda x: 1*x,1)

warranty

In [22]:
#===================
def warranty_yes_function(x):
    if x is None:
        return 0
    elif x.upper().find('OFICIAL')>0 or\
        x.upper() == 'SI' or\
        x.upper().find('ANO')>0 or\
        x.upper().find('FABRICA')>0 or\
        x.upper().find('NUEVO')>0 or\
        x.upper().find('REPUTACION')>0 or\
        x.upper().find('MESES')>0:
        return 1
    else:
        return 0
#===================

pd_data['warranty_yes'] = pd_data['warranty'].apply(warranty_yes_function)


#===================
def warranty_no_function(x):
    if x is None:
        return 0
    elif x.upper().find('USAD')>0 or\
        x.upper() == 'NO' or\
        x.upper().find('VIEJ')>0 or\
        x.upper().find('SIN GARANTIA')>0:
        return 1
    else:
        return 0
#===================

pd_data['warranty_no'] = pd_data['warranty'].apply(warranty_no_function)

official_store_id

In [23]:
pd_data['official_store_id_new'] = pd_data['official_store_id'].apply(lambda x: 0 if np.isnan(x) else 1)

title

In [24]:
pd_data['title_new'] = pd_data['title'].apply(lambda x: 1 if x.upper().find('NUEV')>0 or\
                                                                                     x.upper().find('NEW')>0 or\
                                                                                     x.upper().find('ESTREN')>0\
                                                                                    else 0)

pd_data['title_usado'] = pd_data['title'].apply(lambda x: 1 if x.upper().find('USAD')>0 or\
                                                                x.upper().find('VIEJ')>0 or\
                                                                x.upper().find('ANTIGU')>0\
                                                            else 0)

In [25]:
pd_final = pd.merge(pd_data,pd_no_pago_pivot,on='id',how='left')
pd_final = pd_final.drop(['title','official_store_id','warranty'],axis=1).copy()

In [26]:
# save data
pd_final.to_pickle('./data/data_base.pkl')